In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import codecs
import os
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
import re 

In [7]:
bible_ver_dic = {}

std_bib = pd.read_csv("../data/Bible/t_asv.csv", index_col = [0])
sim_bib = pd.read_csv("../data/Bible/t_bbe.csv", index_col = [0])

std_bib.t = std_bib.t.str.lower()
sim_bib.t = sim_bib.t.str.lower()

def text_cleaner(text):
    newString = re.sub("[\{\[].*?[\}\]]", "", text)
    newString = re.sub('"','', newString)
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newString = newString.lower()
    tokens=newString.split()
    newString=''
    for i in tokens:
        if len(i)>1:                                 
            newString=newString+i+' '  
    return newString

# remove notes from the bible text
std_bib.t = std_bib.t.apply(text_cleaner)
sim_bib.t = sim_bib.t.apply(text_cleaner)

bibles = std_bib.copy()
bibles['std_bib'] = std_bib.t
bibles['sim_bib'] = sim_bib.t
bibles = bibles.drop(columns='t')

np.where(bibles.applymap(lambda x: x == ''))[0]


empty_rows = bibles[bibles['std_bib'] == ''].index
bibles = bibles.drop(empty_rows)
np.where(bibles.applymap(lambda x: x == ''))[0]



bibles = bibles.sample(frac=1)
print(bibles.shape)
bibles.head()
bibles.to_csv('../data/preprocessed/modern_simplefied/bibles_suffled.csv')

train = round(31096 * 0.7)
val = round(31096 * 0.2)
test = round(31096 * 0.1)
print(train, val, test, 'sum:',train+test+val)

(31102, 5)
21767 6219 3110 sum: 31096


In [8]:
train = round(31096 * 0.7)
val = round(31096 * 0.2)
test = round(31096 * 0.1)
print(train, val, test, 'sum:',train+test+val)

21767 6219 3110 sum: 31096


In [9]:
train_df = bibles[0:train].copy()
val_df = bibles[train:val+train].copy()
test_df = bibles[val+train:test+val+train].copy()


In [11]:
train_df['trg'] = train_df['sim_bib']
train_df['src'] = train_df['std_bib']
train_df = train_df.drop(columns = ['b', 'c', 'v', 'sim_bib', 'std_bib'])

val_df['trg'] = val_df['sim_bib']
val_df['src'] = val_df['std_bib']
val_df = val_df.drop(columns = ['b', 'c', 'v', 'sim_bib', 'std_bib'])

test_df['trg'] = test_df['sim_bib']
test_df['src'] = test_df['std_bib']
test_df = test_df.drop(columns = ['b', 'c', 'v', 'sim_bib', 'std_bib'])

In [12]:
train_df.to_csv('../data/preprocessed/modern_simplefied/train_df.csv', header=None)
val_df.to_csv('../data/preprocessed/modern_simplefied/val_df.csv', header=None)
test_df.to_csv('../data/preprocessed/modern_simplefied/test_df.csv', header=None)